In [1]:
import pickle
x_train = pickle.load(open("x_train", "rb"))
x_test = pickle.load(open("x_test", "rb"))
y_train = pickle.load(open("y_train", "rb"))
y_test = pickle.load(open("y_test", "rb"))

max_len = 27
num_word = 26709

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as  np 
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_word)
tokenizer.fit_on_texts(x_train)

train_seq = tokenizer.texts_to_sequences(x_train)
train_padded = pad_sequences(train_seq, maxlen=max_len, padding="post", truncating="post")

train_padded

array([[4344, 2990, 3348, ...,    0,    0,    0],
       [2724, 1020, 2725, ...,    0,    0,    0],
       [3819, 3349, 2085, ...,    0,    0,    0],
       ...,
       [ 204, 2367, 1586, ...,    0,    0,    0],
       [1661,  751,   14, ...,    0,    0,    0],
       [ 796, 1193, 2617, ...,    0,    0,    0]])

In [3]:
test_seq = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_seq, maxlen=max_len, padding="post", truncating="post")

test_padded

array([[ 6079, 20572,  2534, ...,     0,     0,     0],
       [ 3493,   579,  9753, ...,     0,     0,     0],
       [  171,   115, 10999, ...,     0,     0,     0],
       ...,
       [   11,  1541,   568, ...,     0,     0,     0],
       [  169,  3489,   207, ...,     0,     0,     0],
       [ 3200,   767,  2293, ...,     0,     0,     0]])

In [4]:
print(x_train.iloc[0])
print(train_seq[0])

paramount pictures proudly shelves latest film
[4344, 2990, 3348, 2723, 377, 226]


In [15]:
word_index = tokenizer.word_index
print(f"the number of unique words :", len(word_index))
# print(word_index)

the number of unique words : 22847


In [72]:
# word_index

In [74]:
# word_index["film"]

# create the embedding dictionary 

In [6]:
embedding_dic = {}
with open("glove.twitter.27B.100d.txt", "r", encoding="utf8") as f :
    for line in f :
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dic[word] = vectors
f.close()   

In [16]:
embedding_matrix = np.zeros((num_word, 100))
for word, i in tokenizer.word_index.items():
    if i < num_word : 
        emb_vec =embedding_dic.get(word)
        if emb_vec is not None : 
            embedding_matrix[i] = emb_vec

In [17]:
# embedding_dic["paramount"]

In [18]:
# embedding_matrix[1]

In [19]:
word_index["film"]

226

In [21]:
# embedding_dic.get("film")

In [22]:
(embedding_matrix[226] == embedding_dic.get("film")).all()

True

In [98]:
print(train_padded.shape)
print(y_train.shape)

(18696, 27)
(18696,)


In [99]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense,Embedding
from  keras.initializers import Constant

model =  Sequential()
model.add(Embedding(
num_word, 100, embeddings_initializer=Constant(embedding_matrix), input_length=max_len, trainable = False
))
model.add(LSTM(100, dropout=0.1))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics=["accuracy"])

In [100]:
model.fit(train_encode, y_train, validation_data=(test_encode, y_test), epochs=10)

Epoch 1/10
585/585 [==============================] - 14s 19ms/step - loss: 0.5332 - accuracy: 0.7350 - val_loss: 0.8612 - val_accuracy: 0.5315
Epoch 2/10
585/585 [==============================] - 11s 19ms/step - loss: 0.4623 - accuracy: 0.7812 - val_loss: 0.9402 - val_accuracy: 0.4910
Epoch 3/10
585/585 [==============================] - 12s 20ms/step - loss: 0.4235 - accuracy: 0.8032 - val_loss: 0.8538 - val_accuracy: 0.5361
Epoch 4/10
585/585 [==============================] - 11s 20ms/step - loss: 0.3905 - accuracy: 0.8212 - val_loss: 0.9767 - val_accuracy: 0.5044
Epoch 5/10
585/585 [==============================] - 11s 20ms/step - loss: 0.3613 - accuracy: 0.8388 - val_loss: 1.0082 - val_accuracy: 0.5002
Epoch 6/10
585/585 [==============================] - 12s 20ms/step - loss: 0.3342 - accuracy: 0.8546 - val_loss: 1.1603 - val_accuracy: 0.5079
Epoch 7/10
585/585 [==============================] - 12s 20ms/step - loss: 0.3078 - accuracy: 0.8662 - val_loss: 1.1928 - val_accuracy:

In [48]:
# from nltk.tokenize import word_tokenize
# import  nltk 
# def create_corpus(df) : 
#     corpus = []
#     for text in df : 
#         words = [word.lower() for word in word_tokenize(text)]
#         corpus.append(word)
#     return corpus
     

In [49]:
# import pandas as pd 
# a = pd.concat([x_test, x_train], axis=0)


In [50]:
# c = create_corpus(a)
# len(c)